## Import libraries

In [200]:
import fitz
import json
from re import findall
from re import sub
from os import listdir
from os.path import isfile, join
import PyPDF2
import glob
import en_core_web_sm
import numpy as np  
import pandas as pd
import re  
import nltk  
from sklearn.datasets import load_files 
nltk.download('stopwords')  
import pickle  
from nltk.corpus import stopwords
import string


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abdullah.kurkcu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Fontgetter

In [201]:
from operator import itemgetter
def fonts(doc, granularity=False):
    """Extracts fonts and their usage in PDF documents.
    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param granularity: also use 'font', 'flags' and 'color' to discriminate text
    :type granularity: bool
    :rtype: [(font_size, count), (font_size, count}], dict
    :return: most used fonts sorted by count, font style information
    """
    styles = {}
    font_counts = {}

    for page in doc:
        blocks = page.getText("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # block contains text
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if granularity:
                            styles[s['font']] =  {'size':s['size'],'color':s['color']}
                        else:
                            styles[s['font']] =  {'size':s['size'],'color':s['color']}

    return styles

In [192]:
# All the pdf files in the folder
pdf_files = glob.glob("%s/*.pdf" % "../resumesnot/")


# Resumes -> convert to a dictionary with an employee name
resumesTest = pd.DataFrame()

text_dict = {}
# File loop
for file in pdf_files:
    print(file)
    text = ""
    try:
        fileReader = PyPDF2.PdfFileReader(open(file,'rb'))
        count = fileReader.numPages
        text = ""
        i = 0
        while i < count:
            text = fileReader.getPage(i).extractText()
            print(fileReader.getPage(i).getText())
            i = i +1  
            # lowercase
            text = text.lower()
            # remove nubmers
            text = re.sub(r'\d+', '', text)
            # remove punctuation
            text = text.translate(str.maketrans('', '', string.punctuation))
            # remove \n
            text = text.replace('\n',' ')
            # remove ndł
            text= text.replace('ndł',' ')
            # remove \nł
            text = text.replace('\nł',' ')
            # remove ł
            text = text.replace('ł',' ')
            # remove \nł
            text = text.replace('\nł',' ')
            if len(text) > 200:
                ,list(fonts(doc).values())

                text_dict[i] = {'text':text,
                                'fonts':list(fonts(doc).keys()), 
                                'size': [x['size'] for x in list(fonts(doc).values())],
                                'color':[x['color'] for x in list(fonts(doc).values())]
                               }
            else:
                pass
    except Exception as e:
        print(e)
        pass
    


../resumesnot\19.00150_Town of Vail_Gore Valley Trail_Feb19_FINAL_LR.pdf
'PageObject' object has no attribute 'getText'
../resumesnot\19.00150_Town of Vail_Gore Valley Trail_Feb19_FINAL_LR_Part1.pdf
'PageObject' object has no attribute 'getText'
../resumesnot\21.00876 Becker County Heartland Trail Engineering & Design Services - Final.pdf
'PageObject' object has no attribute 'getText'
../resumesnot\21.00980 Sioux City Proposal.pdf
'PageObject' object has no attribute 'getText'


In [199]:
doc = fitz.open("../resumes/17.00532 2.pdf")

styles = {}

for page in doc:
    blocks = page.getText("dict")["blocks"]
    for b in blocks:  # iterate through the text blocks
        if b['type'] == 0:  # block contains text
            for l in b["lines"]:  # iterate through the text lines
                for s in l["spans"]:  # iterate through the text spans
                        styles[s['font']] =  {'size':s['size'],'color':s['color'],'text':s['text']}
styles

{'NewsGothicStd-Bold': {'size': 10.0, 'color': 2236191, 'text': 'Training'},
 'NewsGothicStd': {'size': 10.0,
  'color': 2236191,
  'text': 'Applied Roadway Design'}}

In [188]:
import fitz
doc = fitz.open("../resumes/17.00532 2.pdf")
list(fonts(doc).keys()),list(fonts(doc).values())


(['NewsGothicStd-Bold', 'NewsGothicStd'],
 [{'size': 10.0, 'color': 2236191}, {'size': 10.0, 'color': 2236191}])

In [190]:
[x['size'] for x in list(fonts(doc).values())]

[10.0, 10.0]

In [185]:
from operator import itemgetter
def fonts(doc, granularity=False):
    """Extracts fonts and their usage in PDF documents.
    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param granularity: also use 'font', 'flags' and 'color' to discriminate text
    :type granularity: bool
    :rtype: [(font_size, count), (font_size, count}], dict
    :return: most used fonts sorted by count, font style information
    """
    styles = {}
    font_counts = {}

    for page in doc:
        blocks = page.getText("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # block contains text
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if granularity:
                            styles[s['font']] =  {'size':s['size'],'color':s['color']}
                        else:
                            styles[s['font']] =  {'size':s['size'],'color':s['color']}

    return styles

In [52]:
# All the pdf files in the folder
pdf_files = glob.glob("%s/*.pdf" % "../resumes/")


# Resumes -> convert to a dictionary with an employee name
resumes = pd.DataFrame()

texts = []
# File loop
for file in pdf_files:
    print(file)
    text = ""
    try:
        fileReader = PyPDF2.PdfFileReader(open(file,'rb'))
        count = fileReader.numPages
        text = ""
        i = 0
        while i < count:
            text += fileReader.getPage(i).extractText()
            i = i +1  
        # lowercase
        text = text.lower()
        # remove nubmers
        text = re.sub(r'\d+', '', text)
        # remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))
        # remove \n
        text = text.replace('\n',' ')
        # remove ndł
        text= text.replace('ndł',' ')
        # remove \nł
        text = text.replace('\nł',' ')
        # remove ł
        text = text.replace('ł',' ')
        # remove \nł
        text = text.replace('\nł',' ')

        texts.append(text)
    except Exception as e:
        print(e)
        pass
    
resumes['text'] = texts
resumes['classification'] = "Resume"

../resumes\17.00532 0.pdf
../resumes\17.00532 1.pdf
../resumes\17.00532 2.pdf
../resumes\17.00532 3.pdf
../resumes\18.00308 0.pdf
../resumes\18.00308 1.pdf
../resumes\18.00308 2.pdf
../resumes\18.00308 3.pdf
../resumes\19.00064 0.pdf
../resumes\19.00064 1.pdf
../resumes\19.00064 10.pdf
../resumes\19.00064 11.pdf
../resumes\19.00064 2.pdf
../resumes\19.00064 3.pdf
../resumes\19.00064 4.pdf
../resumes\19.00064 5.pdf
../resumes\19.00064 6.pdf
../resumes\19.00064 7.pdf
../resumes\19.00064 8.pdf
../resumes\19.00064 9.pdf
../resumes\20.00432 0.pdf
../resumes\20.00432 1.pdf
../resumes\20.00432 2.pdf
../resumes\20.00432 3.pdf
../resumes\20.00432 4.pdf
../resumes\20.00973 0.pdf
../resumes\20.00973 1.pdf
../resumes\20.00973 10.pdf
../resumes\20.00973 11.pdf
../resumes\20.00973 12.pdf
../resumes\20.00973 2.pdf
../resumes\20.00973 3.pdf
../resumes\20.00973 4.pdf
../resumes\20.00973 5.pdf
../resumes\20.00973 6.pdf
../resumes\20.00973 7.pdf
../resumes\20.00973 8.pdf
../resumes\20.00973 9.pdf
../resu

In [54]:

resumes.loc[81:117, 'classification']  = 'NotResume'

In [55]:
resumes

,text,classification
0,angy casamento peproject manager angy is a lea...,Resume
1,mike dora peprincipalincharge mike is the mark...,Resume
2,andria schmid peqaqcandria works with the civi...,Resume
3,andria schmid peqaqcandria works with the civi...,Resume
4,angy casamento peproject manager angy is a lea...,Resume
...,...,...
112,project approach,NotResume
113,project approachproject understanding the proj...,NotResume
114,project approachthe project both in terms of t...,NotResume
115,this page is intentionally left blank,NotResume


## Convert text into a vector 

In [96]:
from sklearn.model_selection import train_test_split  
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1,2))


# Do not call fit transform twice because it is stored in vectorizer 
X_train = vectorizer.fit_transform(resumes['text'])
X_test = vectorizer.transform(resumesTest['text'])
y_train = resumes['classification']

## Train Classifier

In [97]:
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)  
text_clf  = classifier.fit(X_train, y_train) 

In [98]:
file = '../downloaded/20.02373 CDOT NPS Western Slope_Ulteig.pdf'
fileReader = PyPDF2.PdfFileReader(open(file,'rb'))
count = fileReader.numPages


results = text_clf.predict(X_test)
resumesTest['results'] = results
results

array(['NotResume', 'NotResume', 'NotResume', 'NotResume', 'NotResume',
       'NotResume', 'NotResume', 'NotResume', 'NotResume', 'NotResume',
       'NotResume', 'NotResume', 'NotResume', 'NotResume', 'NotResume',
       'NotResume', 'NotResume', 'NotResume', 'NotResume', 'NotResume',
       'NotResume', 'NotResume', 'NotResume', 'NotResume', 'NotResume',
       'NotResume', 'NotResume', 'Resume', 'Resume', 'Resume', 'Resume',
       'Resume', 'Resume', 'Resume', 'Resume', 'Resume', 'Resume',
       'Resume', 'Resume', 'Resume', 'Resume', 'Resume', 'Resume',
       'Resume', 'Resume', 'Resume', 'Resume', 'Resume', 'Resume'],
      dtype=object)

In [101]:
resumesTest['text'][48]

' downtown infrastructure reconstruction master plan  city of sioux city  iowa      may       quality  control and  quality  assurance  quality control   in his role as project manager charlie bechtold pe  will be responsible for coordinating activities to provide effective workflow and  meet the deliverable milestones during  each  phase of this project  maintain project schedule   project milestones and tasks will follow a schedule made by  the  ulteig  team  and agreeable to the city of sioux city the team will  develop a detailed project schedule using microsoft project software the project schedule incorporates critical path activi ties for  components of the project and identifies milestone dates for project deliverables ulteig™s  project  panagement staff will also provide  detailed biweekly reports to the assigned individuals  request ing  the reports  maintain central project fil e ulteig will establish a central project file  to become  the repository for all critical  projec

In [86]:
resumesTest['text'][0]

'proposal to perform professional engineering services for  gore valley trail design improvements prepared for town of vail angy casamento peangycasamentoulteigcom       february  '

In [70]:
text = ""
i = 0
X_test = []
while i < count:
    text = fileReader.getPage(i).extractText()
    X_test.append(text)
    
    i = i +1  



X_test = vectorizer.transform(X_test)

## Make a prediction

In [65]:
resumesTest.loc[0]['text']

'proposal to perform professional engineering services for  gore valley trail design improvements prepared for town of vail angy casamento peangycasamentoulteigcom       february  '

In [66]:
resumesTest

,text,classification
0,proposal to perform professional engineering s...,NotResume
1,proposal to perform professional engineering s...,NotResume
2,proposal to perform professional engineering s...,NotResume
3,proposal to perform professional engineering s...,NotResume
4,proposal to perform professional engineering s...,NotResume
...,...,...
56,ulteig sioux city office th street suite ...,NotResume
57,ulteig sioux city office th street suite ...,NotResume
58,ulteig sioux city office th street suite ...,NotResume
59,ulteig sioux city office th street suite ...,NotResume


### OpenCV trials

In [208]:
from pdf2image import convert_from_path

file = '../downloaded/20.02373 CDOT NPS Western Slope_Ulteig.pdf'
images = convert_from_path(file)

PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?